In [1]:
# Load libraries
import os
import sys
import gc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from xgboost import  XGBClassifier
#from hyperopt_func import hyperopt_search  #??????
from hyperopt.pyll import scope
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
import pickle
import scipy

import time
import datetime

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import dill
import shap
import hyperopt



C:\Users\jliu190.000\AppData\Local\Continuum\anaconda3bit64\envs\minipy64\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
xgb.__version__

'0.7'

In [3]:
import sys
print(sys.version)

3.5.4 |Anaconda, Inc.| (default, Nov  8 2017, 14:34:30) [MSC v.1900 64 bit (AMD64)]


In [4]:
def get_plotdata(woe_pipleline,varname):
    """Get plot labels for woe binned variables."""
    
    tmpwoe=woe_pipleline.named_steps['woe']
    plot_data=tmpwoe.encoders[varname].bins[['labels', 'woe', 'obs', 'bins']].copy().drop_duplicates()
    cont_labels = plot_data['labels'].apply(lambda z: not z.startswith('d_'))
    plot_data['plot_bins'] = plot_data['bins'].apply(lambda x: '{:0.2g}'.format(x))
    temp_data = plot_data[cont_labels].copy()
    right_bound = temp_data['plot_bins'].iloc[1:].append(pd.Series(['Inf']))
    temp_data['plot_bins'] = temp_data['plot_bins'].add(' : ').add(list(right_bound))
    plot_data = temp_data.append(plot_data[~cont_labels])
    cont_labels = plot_data['labels'].apply(lambda z: not z.startswith('d_'))
    plot_data['plot_bins'] = np.where(cont_labels, plot_data['plot_bins'], plot_data['labels'])
    return plot_data

In [5]:
def load_data_old(infile="Y:/ml_us2016/data/no.sas7bdat",isupdate="N",
                  old_new_in="N",prodtype=1,all=None):
    """Filter input data by segment and product type or no segmentation at all.
    Decode bytes before printing.
    
    Args:
        Input filename, flag if input data is with new odefind, segment with N/O,
        segment with retail/lease 1/2.
    Returns:
        Dataframe
        """
    sasdata=pd.read_sas(infile)
    
    if isupdate=="N":
        sasdata['poffrdt']=pd.to_timedelta(sasdata.poffrdt, unit='D') + pd.Timestamp('1960-1-1')
        sasdata=decode_byte(sasdata)
        if all is None:
            print ("Loaded segment iupdate:{} infile:{}, old_new:{}, \
                  prod type:{} data segmented?:{}".format(isupdate,infile,old_new_in,prodtype,all))
            sasdata=sasdata[(sasdata.NEW_USED_CD==old_new_in) & (sasdata.o_PROD_TYPE_CD==prodtype) 
            & (sasdata.poffrdt>="2012-01-01")]
        else:
            print ("No segmentation, updated odefind?: {}".format(isupdate))
            sasdata=sasdata[(sasdata.poffrdt>="2012-01-01")]
    
    if isupdate=="Y":
        sasdata['POFFRDT']=pd.to_timedelta(sasdata.POFFRDT, unit='D') + pd.Timestamp('1960-1-1')
        sasdata=decode_byte(sasdata)
        if all is None:
            print ("Loaded segment iupdate:{} infile:{}, old_new:{}, \
                   prod type:{} data segmented?:{}".format(isupdate,infile,old_new_in,prodtype,all))
            sasdata=sasdata[(sasdata.NEW_USED_CD==old_new_in) & (sasdata.PROD_TYPE_CD==prodtype) 
            & (sasdata.POFFRDT>="2012-01-01")]
        else:
            print ("No segmentation, updated odefind?:{}".format(isupdate))
            sasdata=sasdata[(sasdata.POFFRDT>="2012-01-01")]
    return sasdata    

In [6]:
def decode_byte(df):
    """Convert object type variables with byte observations to 'utf-8' encoding.
    
    :param df: input dataframe
    :return: Modified dataframe with byte observations in object cols 
    converted to 'utf-8' encoding.
    """
    objectcols=df.select_dtypes(include=['object']).columns.tolist()
    for col in df.columns.tolist():
        if col in objectcols:
            try:
                df[col]=df[col].apply(lambda x:x.decode() if isinstance(x,(bytes)) else x)
            except AttributeError:
                pass
    return df

In [7]:
# High FICO LEASE NEW with old odefind June 2016
print ("Reading high fico data...")
high_fico=load_data_old(infile=r"W:\GRMOR\Model Innovation\ML1.0\HIGH_FICO_LEASE\Model_Validation\DATA\high.sas7bdat",isupdate="N",old_new_in="N",prodtype=2,all=None)
high_fico.shape
# (25406, 1504)

Reading high fico data...
Loaded segment iupdate:N infile:W:\GRMOR\Model Innovation\ML1.0\HIGH_FICO_LEASE\Model_Validation\DATA\high.sas7bdat, old_new:N,                   prod type:2 data segmented?:None


(25406, 1504)

In [8]:
high_fico.head()

,ln_acctno,porigbr,poffrdt,segment,ODEFIND_30JUN2016,PHWT_30JUN2016,samplingweight,AXZ_PLAN_CD,POP5_006,POP5_007,...,o_odefind_24,o_Thinfile3,o_DECIS_CD,o_purchased,Ford_3seg,dev_flag,cust_rv_m1_no,cust_rv_m2_tn,cust_rv_m4_tk,cust_rv_m5_all
12,000218,CF,2012-11-07,FICO_H,0.0,1.000000,21.899723,NaN,1.0,1.0,...,0.0,0.0,A,1.0,2 THICK,1.0,701.0,729.0,679.0,684.0
14,000227,TF,2014-08-07,FICO_H,1.0,1.277131,1.000000,1,1.0,1.0,...,1.0,0.0,A,1.0,2 THICK,1.0,718.0,728.0,710.0,704.0
15,000230,AE,2012-01-27,FICO_H,1.0,1.000000,1.000000,1,1.0,1.0,...,1.0,0.0,A,1.0,2 THICK,1.0,711.0,731.0,646.0,644.0
21,000246,JK,2012-10-11,FICO_H,1.0,1.000535,1.000000,NaN,1.0,1.0,...,0.0,0.0,A,1.0,2 THICK,1.0,752.0,786.0,797.0,792.0
22,000249,AD,2014-08-12,FICO_H,1.0,2.693890,1.000000,NaN,0.7,0.8,...,1.0,0.0,A,1.0,2 THICK,1.0,703.0,724.0,751.0,740.0


In [9]:
high_fico.NEW_USED_CD.value_counts()

N    25406
Name: NEW_USED_CD, dtype: int64

In [10]:
# sanity check
high_fico.ODEFIND_30JUN2016.value_counts()

0.0    19057
1.0     6349
Name: ODEFIND_30JUN2016, dtype: int64

In [15]:
    # Read updated nofico dataframe odefind march 2017 with updated lnx scores and sampling and phwt.
    print ("Reading updated high fico data...")
    up_highfico=load_data_old(r"W:\GRMOR\Model Innovation\ML1.0\data\DataUsedForTraining\TrainingUpdated2017_3-ODFIND\updt_dat_high_ml.sas7bdat",isupdate="Y",all='Y')
    print ("Merging data...")
    # Merge old and new dataframe with selected updated features only. New dataframe has 
    #lot more observations including non purchase observations.
    # Not using lnx attributes for this excercise.
    new_highfico=pd.merge(high_fico, up_highfico[['ln_acctno',
                                           'ODEFIND_31MAR2017',
                                           'SmplWT_Mar2017',
                                           'PHWT_31MAR2017',
                                           'auto_score',
                                           'cust_rv_m2_tn_v2',
                                           'cust_rv_m5_all_v2',
                                           'cust_rv_m1_no_v2',
                                           'cust_rv_m4_tk_v2']],
                        on='ln_acctno',how='left')

    # Drop old features
    new_highfico.drop(['auto_score_x','cust_rv_m2_tn','cust_rv_m5_all','cust_rv_m1_no',
                    'cust_rv_m4_tk','ODEFIND_30JUN2016','PHWT_30JUN2016',
                    'samplingweight'],1,inplace=True)

    # Rename new updated features after old feature names
    new_highfico.rename(columns={'auto_score_y':'auto_score',
                               'cust_rv_m2_tn_v2':'cust_rv_m2_tn',
                               'cust_rv_m5_all_v2':'cust_rv_m5_all',
                               'cust_rv_m1_no_v2':'cust_rv_m1_no',
                               'cust_rv_m4_tk_v2':'cust_rv_m4_tk',
                               'PHWT_31MAR2017': 'PHWT_30JUN2016',
                               'ODEFIND_31MAR2017':'ODEFIND_30JUN2016',
                               'SmplWT_Mar2017':'samplingweight'},inplace=True)
    print ("Done!")

Reading updated high fico data...
No segmentation, updated odefind?:Y
Merging data...
Done!


In [16]:
if True:
    assert(new_highfico.shape[0]==high_fico.shape[0])

In [17]:
print(up_highfico.shape)

(70365, 1517)


In [18]:
df=high_fico.copy()

#### Build list of variables to exclude from analysis

In [19]:
def df_status(df):
    """For each variable it returns: Quantity and percentage of zeros 
    (q_zeros and p_zeros respectevly). Same metrics for NA values (q_NA/p_na). 
    Next two columns indicates data type and quantity of unique values. 
    Last two columns indicate max and min values.
    This function print and return the results.
    """  
    print ("Total observations in data {}:".format(df.shape[0]))
    print ("Total columns in data {}:".format(df.shape[1]))

    cat_cols=df.select_dtypes(include=['object']).shape[1]
    print ("Total categorical/object columns in data {}:".format(cat_cols))
    
    date_cols=df.filter(like='date').columns.tolist()
    if len(date_cols)>0:
        print ("Date columns found {}:".format(date_cols))
        print ("Consider converting these cols to datetime format using pd.todatetime..")
        
    status=pd.concat([
           df.apply(lambda x:(x.values==0).sum()),
           df.apply(lambda x:round( (x.values==0).sum()/float(df.shape[0])*100,2)),\
           df.isnull().sum(),
           np.round((df.isnull().sum()/float(df.shape[0]))*100,2),\
           df.dtypes,
           df.apply(lambda x:len(x.unique()))],
           #df.apply(lambda x:x.max()),
           #df.apply(lambda x:x.min())],
        axis=1,
          keys=['num_zeros','pct_zeros','num_na','pct_na','type','unique_vals'])
        #'max','min'])
    #if (status.type=="object" ) & (status.num_na>0):
    #    print "there are missing values in object column"
    return status

In [20]:
# Compute the data summary
status=df_status(high_fico)

Total observations in data 25406:
Total columns in data 1504:
Total categorical/object columns in data 72:
Date columns found ['inputprovideddateofbirth', 'ssndatelowissued']:
Consider converting these cols to datetime format using pd.todatetime..


In [21]:
# Variables with more than 99 percent missing values.
miss_99_vars=status[status.pct_na>=99].index.tolist()
print ("Number of vars with 99% missing values: {}".format(len(miss_99_vars)))
print (miss_99_vars)

Number of vars with 99% missing values: 41
['POP5_224', 'POP5_416', 'POP5_417', 'POP5_418', 'POP5_419', 'KBS3_041', 'KBS3_042', 'DSA2_105', 'DSA2_128', 'DSA2_321', 'DSA2_346', 'DSA2_347', 'DSA2_348', 'DSA2_367', 'DSA2_377', 'DSA2_441', 'DSA2_471', 'DSA2_472', 'DSA_046', 'DSA_079', 'FIN_WHSL_PC', 'IDA_CR_OPTC_SCR_NB', 'LOG_ADVNC_PC', 'LOG_NET_ADVNC_PC', 'ADVNC_AM', 'DCASHPR', 'OCPOIND', 'auto_reason5', 'bankcard_reason5', 'short_term_lending_reason5', 'telecommunications_reason5', 'alert1', 'alert2', 'alert3', 'alert4', 'alert5', 'alert6', 'alert7', 'alert8', 'alert9', 'alert10']


In [22]:
# Invariant variables
constant_vars=status[status.unique_vals==1].index.tolist()
print ("Number of invariant vars: {}".format(len(constant_vars)))
print (constant_vars)

Number of invariant vars: 34
['segment', 'KBS5_008', 'KBS3_041', 'KBS3_042', 'FPOP_041', 'FPOP_043', 'FPOP_048', 'DSA2_321', 'DSA_046', 'DSA_079', 'CB_RPT_TYPE_CD', 'FIN_WHSL_PC', 'NEW_USED_CD', 'VEH_AGE_NB', 'LOG_ADVNC_PC', 'LOG_NET_ADVNC_PC', 'DDRBTAM', 'ADVNC_AM', 'DCASHPR', 'OCPOIND', 'auto_reason5', 'bankcard_reason5', 'alert2', 'alert3', 'alert4', 'alert5', 'alert6', 'alert7', 'alert8', 'alert9', 'alert10', 'o_PROD_TYPE_CD', 'o_DECIS_CD', 'o_purchased']


In [23]:
# Remove vars for business reasons
# Reason codes should not used + other leak vars
buisness_vars=['VEHMODEL','ssndeceased','mkey','APR_SUBV',
               'RSDNC_CD','RSDNC_MO_NB',
               'OCPOIND',
               'IDA_CR_OPTC_SCR_NB',
              # 'LOG_TOT_PMT_NET_WHSL_PC', # only RCL
               #'LOG_TOTPMT_WHSL_TATT_PC',  # Only RCL
               #'TOT_PMT_WHSL_PC',
               'ADVNC_TATT_PC', # retail vars
               'ADVNC_TATT_PC', # retail vars
               'assetindexprimaryfactor',
               'subjectabilityprimaryfactor','subjectwillingnessprimaryfactor',
               'subjectstabilityprimaryfactor','porigbr','ssndatelowissued',
               'ssnsubjectcount', # SSN vars
               'alert1','subjectage','POP_127','KBS3_103','APPLCNT_AGE_NB',
               'CBYR_APPLCNT_AGE_NB','ORG_206','ECA_007',"DSA_100",'DSA_101'] + \
df.filter(like="_reason",axis=1).columns.tolist()+\
["ETC_009",'OVF_007',"OVF_008","OVF_036","DBT_016",
 "DBT_060",'DSA2_123','OCC_008',"MO_ON_JOB_BUS_NB",
'o_odefind_24','dev_flag','ln_acctno'] # these added after sitting on logistic model meetings
# vars legal did not like
#'DBT_051'

#print (buisness_vars)
print ("{}".format(len(buisness_vars)))

59


In [24]:
buisness_vars

['VEHMODEL',
 'ssndeceased',
 'mkey',
 'APR_SUBV',
 'RSDNC_CD',
 'RSDNC_MO_NB',
 'OCPOIND',
 'IDA_CR_OPTC_SCR_NB',
 'ADVNC_TATT_PC',
 'ADVNC_TATT_PC',
 'assetindexprimaryfactor',
 'subjectabilityprimaryfactor',
 'subjectwillingnessprimaryfactor',
 'subjectstabilityprimaryfactor',
 'porigbr',
 'ssndatelowissued',
 'ssnsubjectcount',
 'alert1',
 'subjectage',
 'POP_127',
 'KBS3_103',
 'APPLCNT_AGE_NB',
 'CBYR_APPLCNT_AGE_NB',
 'ORG_206',
 'ECA_007',
 'DSA_100',
 'DSA_101',
 'auto_reason1',
 'auto_reason2',
 'auto_reason3',
 'auto_reason4',
 'auto_reason5',
 'bankcard_reason1',
 'bankcard_reason2',
 'bankcard_reason3',
 'bankcard_reason4',
 'bankcard_reason5',
 'short_term_lending_reason1',
 'short_term_lending_reason2',
 'short_term_lending_reason3',
 'short_term_lending_reason4',
 'short_term_lending_reason5',
 'telecommunications_reason1',
 'telecommunications_reason2',
 'telecommunications_reason3',
 'telecommunications_reason4',
 'telecommunications_reason5',
 'ETC_009',
 'OVF_007',
 'OVF_008',
 'OVF_036',
 'DBT_016',
 'DBT_060',
 'DSA2_123',
 'OCC_008',
 'MO_ON_JOB_BUS_NB',
 'o_odefind_24',
 'dev_flag',
 'ln_acctno']

In [25]:
# Lexis nexis variables in the data
lnxvars=[var for var in df.columns if var.islower() 
         and var not in buisness_vars+miss_99_vars+constant_vars+ ['poffrdt',
                                                                   'segment',
                                                                   'samplingweight', 
                                                                   'auto_score', 
                                                                   'cust_rv_m2_tn',
                                                                   'cust_rv_m5_all']]
print (len(lnxvars))

186


In [26]:
#lnxvars

In [27]:
high=pd.read_csv(r"W:\GRMOR\Model Innovation\ML1.0\HIGH_FICO_LEASE\Model_Validation\from_sean\MASTER_VARLIST_HIGH.csv")

highvars=(high[((high.Type=="FORD") | (high.Type=="LN")) & ((high.Step==1)  | (high.Step==2)  | 
                                (high.Type=="TARGET"))]['Variable'].values.tolist())

print ("Keep {} variables from low fico segment".format(len(highvars)))

highvars=[var.split("WOE_")[1] for var in highvars]

print (len(highvars))

#j
#Keep 1232 variables from low fico segment

Keep 1232 variables from low fico segment
1232


In [28]:
# Master list of keep variables that are either NOT business vars, missing 99 percent 
# data and/or are contant variables
keepvars=[var for var in highvars if var not in buisness_vars+miss_99_vars+
          constant_vars+lnxvars]
print ("Keep {} variables for modeling".format(len(keepvars)))

Keep 1065 variables for modeling


In [29]:
# Keep only selected vars (overwrite original dataframe)
# Overwrite the original data frame 

df=df[keepvars+['ODEFIND_30JUN2016','poffrdt','segment','PHWT_30JUN2016','samplingweight']]
print (df.shape)
#(25406, 1070)

(25406, 1070)


#### Variable treatment:

In [30]:
from sklearn.base import TransformerMixin,BaseEstimator

class occ_codefix(TransformerMixin):
    """We exclude certain occupations from being available. 
    66	FMC-00066	disability
    94	FMC-00094	homemaker
    96	FMC-00096	housewife
    154	FMC-00154	retired
    165	FMC-00165	social-security
    193	FMC-00193	unemployed
    """
    def fit(self,X,y=None):
        return self

    def transform(self, X):
        """Transform dataframe."""
        output = X.copy()
        output['KBS1_183']=output['KBS1_183'].replace({66:np.nan,94:np.nan,
                96:np.nan,154:np.nan,165:np.nan,193:np.nan},inplace=False)
        return output

In [31]:
# Special Codes (sean provided from his enterprise miner output)
# We will use the information for WoE transformation for some variables
codes=pd.read_excel(r"W:\GRMOR\Model Innovation\ML1.0\HIGH_FICO_LEASE\Model_Validation\from_sean\ign_codes_latest.xlsx")

In [32]:
# Some vars names in the codes file have to fixed because SAS truncates long names.
# Fix var names (SAS issue)
# Some vars have long names. SAS truncates names at 32 char. (Sean provided)
fix_ign=pd.read_excel(r"W:\GRMOR\Model Innovation\ML1.0\HIGH_FICO_LEASE\Model_Validation\from_sean\fix_ign.xlsx")

In [33]:
fix_ign

,rename,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,AddrCurrentAVMRatio12MonthPrior,=,var_1,1,NaN
1,AddrCurrentAVMRatio60MonthPrior,=,var_2,2,NaN
2,AddrCurrentAVMValue12Month,=,var_3,3,ADDRCURRENTAVMRATIO12MONTHPRIOR ADDRCURRENTAVM...
3,AddrCurrentAVMValue60Month,=,var_4,4,NaN
4,AddrCurrentLastSalesPrice,=,var_5,5,var_1 var_2 var_3 var_4 var_5 var_6 var_7 var_...
5,AddrCurrentTaxMarketValue,=,var_6,6,NaN
6,AddrInputAVMRatio12MonthPrior,=,var_7,7,NaN
7,AddrInputAVMRatio60MonthPrior,=,var_8,8,NaN
8,AddrInputAVMValue12Month,=,var_9,9,NaN
9,AddrInputAVMValue60Month,=,var_10,10,NaN
